In [2]:
import mlflow
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [3]:
# Set the tracking URI to the local mlruns directory
mlflow.set_tracking_uri("file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns")

# Create and set the experiment
experiment_name = "02_model_improvement_TFIDF(1,3)_max_features"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='file:///Users/mustainbillah/Projects/sentiment-analysis/mlruns/845939112060557627', creation_time=1748570823142, experiment_id='845939112060557627', last_update_time=1748570823142, lifecycle_stage='active', name='02_model_improvement_TFIDF(1,3)_max_features', tags={}>

In [4]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [5]:
def run_experiment(vectorizer_max_features):
    ngram_range = (1, 3)

    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=vectorizer_max_features)

    
    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    with mlflow.start_run() as run:
        mlflow.set_tag('mlflow.runName', f'TFIDF_Trigrams_max_features_{max_features}')
        mlflow.set_tag('experiment_type', 'feature_engineering')
        mlflow.set_tag('model_type', 'RandomForestClassifier')

        mlflow.set_tag('Description', f'Random Forest model with TF-IDF Trigrams, max_features: {vectorizer_max_features}')

        mlflow.log_param('vectorizer_type', 'TF-IDF')
        mlflow.log_param('ngram_range', ngram_range)
        mlflow.log_param('vectorizer_max_features', vectorizer_max_features)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        mlflow.log_metric('accuracy', accuracy_score(y_test, y_pred))

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f'{label}_{metric}', value)
        
        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 6))
        sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix: TF-IDF Trigrams, max_features={vectorizer_max_features}')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.savefig(f'{vectorizer_max_features}_confusion_matrix.png')
        plt.close()
        mlflow.log_artifact(f'{vectorizer_max_features}_confusion_matrix.png')
        
        # log the model
        mlflow.sklearn.log_model(model, f'random_forest_tfidf_trigrams{vectorizer_max_features}')

# Run experiments for different max_features

max_features = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_feature in max_features:
    run_experiment(max_feature)




2025/05/30 08:23:47 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/30 08:23:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/30 08:24:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/30 08:24:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/30 08:24:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/05/30 08:24:23 